In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors 
from shapely.geometry import Point, shape, Polygon, MultiPolygon
from shapely import geometry
from shapely.ops import unary_union
from srai.loaders.osm_loaders import OSMOnlineLoader
from srai.loaders.osm_loaders.filters.popular import get_popular_tags
from functional import seq
from matplotlib import colormaps
from srai.constants import WGS84_CRS
from srai.plotting.folium_wrapper import plot_regions
from srai.regionalizers import VoronoiRegionalizer, geocode_to_region_gdf, AdministrativeBoundaryRegionalizer
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors 

import cartopy.crs as ccrs
import cartopy.feature as cfeature


import branca.colormap as cm_branca
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from analysis_functions import  make_PDF, make_lognormal_PDF
import voronoi_functions as vf 

from datetime import datetime, timedelta
import xarray as xr

# set plotscale
plt.style.use('../python_style_Meike.mplstyle')

In [ ]:
# settings
particle_types = ['tracer','inertial_SM_drag_Rep','inertial_SM_Rep_constant']
coriolislist = [True,False]
B =B = 0.68
tau = 2994.76
T = 719 
gradient = True
runtime = timedelta(days=30)
starttime = datetime(2023, 9, 1)
endtime = starttime + runtime
displacement = 300
loc = "NWES"
land_handling = "anti_beaching"
Rep = 0
Replist = [0,10,100,1000]



base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'

basefile_tracer =outputfile_tracer = (
      base_directory + "{particle_type}/voronoi_data/{loc}_"
      "start{y_s:04d}_{m_s:02d}_{d_s:02d}_T{T:04d}h.geojson"
)

basefile_tracer_random = (
      base_directory + "{particle_type}/voronoi_data/{loc}_"
      "start{y_s:04d}_{m_s:02d}_{d_s:02d}_T{T:04d}h_{d_s:02d}.parquet"
)

basefile_MR = (
      base_directory + "{particle_type}/voronoi_data/{loc}_"
      "start{y_s:04d}_{m_s:02d}_{d_s:02d}_T{T:04d}h_"
      "B{B:04d}_tau{tau:04d}_cor_{coriolis}_gradient_{gradient}.geojson"
)

basefile_MR_Rep_constant = (
      base_directory + "{particle_type}/voronoi_data/{loc}_"
      "start{y_s:04d}_{m_s:02d}_{d_s:02d}_T{T:04d}h_Rep_{Rep:04d}"
      "B{B:04d}_tau{tau:04d}_cor_{coriolis}_gradient_{gradient}.geojson"
)

inputfiles = {'tracer':basefile_tracer,
              'inertial_SM_drag_Rep':basefile_MR,
              'inertial_SM_Rep_constant':basefile_MR_Rep_constant,
              }

In [ ]:
# settings 
data = {}
for pt in particle_types:
    data[pt]={}
    if(pt =='tracer'):
        data[pt][True]={}
        data[pt][True][None]={}
    else:
        for coriolis in coriolislist:
            data[pt][coriolis]={}

for pt in particle_types:
    if(pt == 'tracer'):
        inputfile = inputfiles[pt].format(
            loc=loc,
            y_s=starttime.year,
            m_s=starttime.month,
            d_s=starttime.day,
            y_e=endtime.year,
            m_e=endtime.month,
            d_e=endtime.day,
            land_handling=land_handling,
            particle_type=pt,
            T=T
        )
        data[pt][True][None] = gpd.GeoDataFrame.from_file(inputfile)
    elif(pt == 'inertial_SM_drag_Rep'):
        for coriolis in coriolislist:
            inputfile = inputfiles[pt].format(
                loc=loc,
                y_s=starttime.year,
                m_s=starttime.month,
                d_s=starttime.day,
                y_e=endtime.year,
                m_e=endtime.month,
                d_e=endtime.day,
                land_handling=land_handling,
                coriolis=coriolis,
                particle_type=pt,
                d = displacement,
                Rep = Rep,
                gradient = gradient,
                B = int(1000*B),
                tau = int(tau),
                T=T
            )
            data[pt][coriolis][None] = gpd.GeoDataFrame.from_file(inputfile)
    else:
        for Rep in Replist:
            inputfile = inputfiles[pt].format(
                loc=loc,
                y_s=starttime.year,
                m_s=starttime.month,
                d_s=starttime.day,
                y_e=endtime.year,
                m_e=endtime.month,
                d_e=endtime.day,
                land_handling=land_handling,
                coriolis=True,
                particle_type=pt,
                d = displacement,
                Rep = Rep,
                gradient = gradient,
                B = int(1000*B),
                tau = int(tau),
                T=T
            )
            data[pt][True][Rep] = gpd.GeoDataFrame.from_file(inputfile)


In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()},figsize=(12,8))
ax.add_feature(cfeature.LAND, edgecolor="black", color="lightgray")
vf.plot_voronoi(fig, ax , data['tracer'][True][None],colormap = cm.Blues,
             colormap_scale = 'log',
             color_scale_type = 'density',
             vmin = 1E-3,
             vmax = 1E1, 
             colorbar_on =False)

# gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#             linewidth=0, color='gray', alpha=0.5, linestyle='--')
# gl.top_labels = False
# gl.right_labels = False
# gl.xlabel_style = {'size': 15}
# gl.ylabel_style =  {'size': 15}
ax.set_frame_on(False)
fig.tight_layout()
fig.savefig('../figures/poster/voronoi_tesselation_NWES.pdf')


In [ ]:

fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()},figsize=(14,8))
ax.add_feature(cfeature.LAND, edgecolor="black", color="lightgray")
vf.plot_voronoi(fig, ax , data['inertial_SM_drag_Rep'][True][None],colormap = cm.Blues,
             colormap_scale = 'log',
             color_scale_type = 'density',
             vmin = 1E-3,
             vmax = 1E1)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 15}
gl.ylabel_style =  {'size': 15}
ax.set_xlim(-15,9)
ax.set_ylim(46.5,60.8)

In [ ]:
Rep = 0
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax.add_feature(cfeature.LAND, edgecolor="black", color="lightgray")
vf.plot_voronoi(fig, ax , data['inertial_SM_Rep_constant'][True][Rep],colormap = cm.Blues,
             colormap_scale = 'log',
             color_scale_type = 'density',
             vmin = 1E-3,
             vmax = 1E1)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 15}
gl.ylabel_style =  {'size': 15}


In [ ]:
Blues = colormaps['Blues']



# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.4, 0.9, 3))

fig, ax = plt.subplots()

density = data['tracer'][True][None]['density'].values
bins_tracer, pdf_tracer = make_lognormal_PDF(density,norm=True, nbins = 100,vmin=1E-3,vmax=1E4)
ax.plot(bins_tracer,pdf_tracer,'-',color='black',zorder=-20)

density = data['inertial_SM_Rep_constant'][True][10]['density'].values
bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E4)
ax.plot(bins,pdf,'--',color=bluelist[0])

density = data['inertial_SM_Rep_constant'][True][100]['density'].values
bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E4)
ax.plot(bins,pdf,'--',color=bluelist[1])

density = data['inertial_SM_Rep_constant'][True][1000]['density'].values
bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E4)
ax.plot(bins,pdf,'--',color=bluelist[2])

ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('PDF')
# ax.set_xlim(0,0.1)
# ax.set_yscale('log')
ax.legend(['tracer','Re$_p$ = 10','Re$_p$ = 100','Re$_p$ = 1000'])

In [ ]:
Blues = colormaps['Blues']



# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.4, 0.9,2))

fig, ax = plt.subplots(figsize=(10,5))

density = data['tracer'][True][None]['density'].values
bins_tracer, pdf_tracer = make_lognormal_PDF(density,norm=True, nbins = 100,vmin=1E-3,vmax=1E6)
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf_tracer = np.cumsum(pdf_tracer)
# ax.plot(bins,cdf_tracer,'-',color='black',zorder=-20)

density = data['inertial_SM_Rep_constant'][True][100]['density'].values
bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[0])

# density = data['inertial_SM_Rep_constant'][True][100]['density'].values
# bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
# bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
# bin_width = np.diff(bin_edges)
# cdf = np.cumsum(pdf)
# ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[1])

# density = data['inertial_SM_Rep_constant'][True][1000]['density'].values
# bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
# bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
# bin_width = np.diff(bin_edges)
# cdf = np.cumsum(pdf)
# ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[2])

density = data['inertial_SM_drag_Rep'][True][None]['density'].values
bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[1])

# density = data['inertial_SM_drag_Rep'][False][None]['density'].values
# bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
# bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
# bin_width = np.diff(bin_edges)
# cdf = np.cumsum(pdf)
# ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[2])


ax.set_xscale('log')
ax.set_xlabel('Particle density [1/km$^2$]')
ax.set_ylabel('CDF - tracer CDF')
ax.legend(['Re$_p$ =100','Re$_p$ flexible'],handlelength=1.5)

fig.tight_layout()
fig.savefig('../figures/poster/CDF.pdf')

In [ ]:
Blues = colormaps['Blues']



# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.4, 0.9,2))

fig, ax = plt.subplots()

density = data['tracer'][True][None]['density'].values
bins_tracer, pdf_tracer = make_lognormal_PDF(density,norm=True, nbins = 100,vmin=1E-3,vmax=1E6)
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf_tracer = np.cumsum(pdf_tracer)
ax.plot(bins_tracer,cdf_tracer,'-',color='black',zorder=-20)

density = data['inertial_SM_Rep_constant'][True][100]['density'].values
bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf,'--',color=bluelist[0])

# density = data['inertial_SM_Rep_constant'][True][100]['density'].values
# bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
# bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
# bin_width = np.diff(bin_edges)
# cdf = np.cumsum(pdf)
# ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[1])

# density = data['inertial_SM_Rep_constant'][True][1000]['density'].values
# bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
# bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
# bin_width = np.diff(bin_edges)
# cdf = np.cumsum(pdf)
# ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[2])

density = data['inertial_SM_drag_Rep'][True][None]['density'].values
bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf,'--',color=bluelist[1])

# density = data['inertial_SM_drag_Rep'][False][None]['density'].values
# bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
# bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
# bin_width = np.diff(bin_edges)
# cdf = np.cumsum(pdf)
# ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[2])


ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('CDF')

In [ ]:

#1E-3,vmax=1E6
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf = np.cumsum(pdf_tracer)
fig,ax=plt.subplots()
ax.plot(bins_tracer,cdf)
ax.set_xscale('log')

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax.add_feature(cfeature.LAND, edgecolor="black", color="lightgray")
vf.plot_voronoi(fig, ax , data['tracer'][True][None],colormap = cm.Blues,
             colormap_scale = 'log',
             color_scale_type = 'density',
             vmin = 1E-3,
             vmax = 1E1)

# gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#             linewidth=0, color='gray', alpha=0.5, linestyle='--')
# gl.top_labels = False
# gl.right_labels = False
# gl.xlabel_style = {'size': 18}
# gl.ylabel_style =  {'size': 18}
ax.set_xlim(-14.6,-10.5)
ax.set_ylim(54.4,56.1)
ax.set_frame_on(False)
fig.tight_layout()
fig.savefig('../figures/poster/voronoi_tesselation_zoomed.pdf')

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax.add_feature(cfeature.LAND, edgecolor="black", color="lightgray")
vf.plot_voronoi(fig, ax , data['inertial_SM_Rep_constant'][True][100],colormap = cm.Blues,
             colormap_scale = 'log',
             color_scale_type = 'density',
             vmin = 1E-3,
             vmax = 1E1)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}
# ax.set_xlim(-14.6,-10.5)
# ax.set_ylim(54.4,56.1)
ax.set_xlim(-4,2)
ax.set_ylim(48.5,51.5)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax.add_feature(cfeature.LAND, edgecolor="black", color="lightgray")
vf.plot_voronoi(fig, ax , data['inertial_SM_drag_Rep'][True][None],colormap = cm.Blues,
             colormap_scale = 'log',
             color_scale_type = 'density',
             vmin = 1E-3,
             vmax = 1E1)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}
ax.set_xlim(-14.6,-10.5)
ax.set_ylim(54.4,56.1)